In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from qids_package.qids import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from qids_package.qids import *
import warnings
# from submit import submit
import matplotlib.pyplot as plt
import copy

from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")

In [2]:
seed = 257248
stock_num = 54
train_day_num_total = 1000
train_day_num = 1000 - 2
test_day_num = 700
timeslot_num = 50

In [3]:
calc_log = lambda df: np.log(np.where(df > 1e-8, df, 1e-8))
calc_mean = lambda df: df.mean(axis=0)
calc_max = lambda df: df.max(axis=0)
calc_min = lambda df: df.min(axis=0)
calc_std = lambda df: df.std()
calc_var = lambda df: df.var()
calc_add = lambda df1, df2: df1 + df2
calc_diff = lambda df1, df2: df1 - df2
calc_prod = lambda df1, df2: df1 * df2
calc_div = lambda df1, df2: df1 / df2

In [4]:
def preprocess(fun, mar, ret=None):
    fun["stock_id"] = fun["date_time"].apply(lambda x: x.split("d")[0][1:]).astype("int")
    fun["day"] = fun["date_time"].apply(lambda x: x.split("d")[1][:]).astype("int")
    # fun["log_pb"] = calc_log(fun["pb"])
    # fun["log_ps"] = calc_log(fun["ps"])
    fun = fun.sort_values(by=["stock_id", "day"])
    na_fun = fun.loc[fun["day"].isin([999, 1000])]
    fun = fun.drop(na_fun.index, axis=0).reset_index(drop=True)
    na_fun = na_fun.reset_index(drop=True)

    mar["stock_id"] = mar["date_time"].apply(lambda x: x.split("d")[0][1:]).astype("int")
    mar["day"] = mar["date_time"].apply(lambda x: x.split("d")[1].split("p")[0]).astype("int")
    mar["time"] = mar["date_time"].apply(lambda x: x.split("p")[1]).astype("int")
    mar = mar.sort_values(by=["stock_id", "day", "time"]).reset_index(drop=True)
    na_mar = mar.loc[mar["day"].isin([999, 1000])]
    mar = mar.drop(na_mar.index, axis=0).reset_index(drop=True)
    na_mar = na_mar.reset_index(drop=True)

    combined = copy.deepcopy(fun)
    if ret is not None:
        ret["stock_id"] = ret["date_time"].apply(lambda x: x.split("d")[0][1:]).astype("int")
        ret["day"] = ret["date_time"].apply(lambda x: x.split("d")[1][:]).astype("int")
        # ret["log_pb"] = calc_log(ret["pb"])
        # ret["log_ps"] = calc_log(ret["ps"])
        ret = ret.sort_values(by=["stock_id", "day"]).reset_index(drop=True)
        combined["return"] = ret["return"]
        day_num = train_day_num
    else:
        day_num = test_day_num

    mar_summary = []
    start = 0
    for stock in range(stock_num):
        end = start + day_num * timeslot_num
        stock_info = mar.iloc[start:end, :]
        day_start = 0
        for day in range(day_num):
            day_end = day_start + timeslot_num
            stock_info_per_day = stock_info.iloc[day_start:day_end, :]
            mar_summary.append([
                calc_mean(stock_info_per_day["open"]),
                calc_mean(stock_info_per_day["close"]),
                calc_mean(stock_info_per_day["high"]),
                calc_mean(stock_info_per_day["low"]),
                calc_mean(stock_info_per_day["volume"]),
                calc_mean(stock_info_per_day["money"]),
                calc_max(stock_info_per_day["high"]),
                calc_max(stock_info_per_day["volume"]),
                calc_max(stock_info_per_day["money"]),
                calc_min(stock_info_per_day["low"]),
                #calc_min(stock_info_per_day["volume"]),
                #calc_min(stock_info_per_day["money"]),
                #calc_std(stock_info_per_day["volume"]),
                #calc_std(stock_info_per_day["money"]),
                #calc_var(stock_info_per_day["volume"]),
                #calc_var(stock_info_per_day["money"]),
                calc_max(calc_div(calc_diff(stock_info_per_day["close"], stock_info_per_day["open"]), stock_info_per_day["open"])),
                calc_max(calc_div(calc_diff(stock_info_per_day["high"], stock_info_per_day["low"]), stock_info_per_day["open"])),
            ])
            day_start = day_end
        start = end
    cols = [
        "open_mean",
        "close_mean",
        "high_mean",
        "low_mean",
        "volume_mean",
        "money_mean",
        "high_max",
        "volume_max",
        "money_max",
        "low_min",
        #"volume_min",
        #"money_min",#
        #"volume_std",#
        #"money_std",#
        #"volume_var",#
        #"money_var",#
        "price_diff",
        "price_diff_max",
    ]
    mar_summary = pd.DataFrame(mar_summary, columns=cols)
    combined = pd.concat([combined, mar_summary], axis=1)

    return [combined, fun, mar, na_fun, na_mar, ret] if ret is not None else [combined, fun, mar, na_fun, na_mar]

In [5]:
ROOT_PATH = "/kaggle/input/hku-qids-2023-quantitative-investment-competition/"
TRAIN_MARKET_PATH = f'{ROOT_PATH}first_round_train_market_data.csv'
# TRAIN_MARKET_PATH = f'{ROOT_PATH}train.csv'
TRAIN_FUNADMENTAL_PATH = f'{ROOT_PATH}first_round_train_fundamental_data.csv'
TRAIN_RETURN_PATH = f'{ROOT_PATH}first_round_train_return_data.csv'

TEST_ROOT_PATH = "/kaggle/input/hku-qids-2023-quantitative-investment-competition/qids_package/"
TEST_MARKET_PATH = f'{TEST_ROOT_PATH}first_round_test_market_data.csv'
# TEST_MARKET_PATH = f'{ROOT_PATH}test.csv'
TEST_FUNADMENTAL_PATH = f'{TEST_ROOT_PATH}first_round_test_fundamental_data.csv'

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 350)

#read data
df_train_market = pd.read_csv(TRAIN_MARKET_PATH)
df_train_return = pd.read_csv(TRAIN_RETURN_PATH)
df_train_fundamental = pd.read_csv(TRAIN_FUNADMENTAL_PATH)

# df_test_market = pd.read_csv(TEST_MARKET_PATH)
# df_test_fundamental = pd.read_csv(TEST_FUNADMENTAL_PATH)

In [6]:
def reorder(df):
    df_cols = df.columns
    if 'return' not in df_cols:
        df_cols_prior = ['date_time', 'stock_id', 'day']
    else:
        df_cols_prior = ['date_time', 'stock_id', 'day', 'return']
    for col in df_cols:
        if col not in df_cols_prior:
            df_cols_prior.append(col)
    if 'return' in df_cols_prior:
        df_cols_prior.remove('return')
        df_cols_prior.append('return')
    return df[df_cols_prior]

In [7]:
train_combined, train_fun, train_mar, train_na_fun, train_na_mar, train_ret = preprocess(df_train_fundamental, df_train_market, df_train_return)

In [8]:
train = reorder(train_combined)
train_y = train["return"]
train = train.drop(columns=["return", "date_time",'stock_id','day'])
# test = test.drop(columns=["date_time",'stock_id','day'])
train

,turnoverRatio,transactionAmount,pe_ttm,pe,pb,ps,pcf,open_mean,close_mean,high_mean,low_mean,volume_mean,money_mean,high_max,volume_max,money_max,low_min,price_diff,price_diff_max
0,3.6794,17229.0,34.4425,32.3029,4.9425,3.8180,-578.7700,24.550554,24.536720,24.578718,24.508076,454075.84,1.114925e+07,25.1378,1234546.0,3.056567e+07,24.2396,0.006510,0.007816
1,3.2535,18378.0,33.3198,31.2498,4.7814,3.6935,-559.9031,23.955800,23.938318,23.988574,23.907490,401517.02,9.601600e+06,24.5552,853209.0,2.052481e+07,23.5234,0.004981,0.010720
2,2.4947,13686.0,23.3887,23.3887,4.3823,2.9722,-42.8676,23.626130,23.628554,23.661090,23.591664,307969.24,7.275962e+06,23.7783,708274.0,1.677659e+07,23.3899,0.007227,0.007741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53889,0.8417,5204.0,10.1469,10.1469,1.9012,0.3880,128.1758,9.436804,9.440208,9.455750,9.424180,164545.76,1.554019e+06,9.5162,615056.0,5.839099e+06,9.2613,0.006528,0.013057
53890,1.4411,9143.0,10.4471,10.4471,1.9574,0.3995,131.9680,9.811140,9.816240,9.844646,9.790510,281720.62,2.775468e+06,10.1838,2456106.0,2.435143e+07,9.5041,0.021176,0.032380
53891,1.2933,9681.0,10.6513,10.6513,1.9956,0.4073,134.5467,9.820860,9.827900,9.850964,9.793908,252827.46,2.483025e+06,9.9896,894193.0,8.680005e+06,9.6012,0.014920,0.017410


In [9]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
score_lst = []
reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=10000, tol=1e-6,penalty='l1',random_state=36400))
reg.fit(train, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdregressor',
                 SGDRegressor(max_iter=10000, penalty='l1', random_state=36400,
                              tol=1e-06))])

In [10]:
env = make_env()  # initialize the environment

Environment is initialized.


In [ ]:
# sample code using random number as prediction
import random
random.seed(42)
while not env.is_end():
    fundamental_df, market_df = env.get_current_market()  # get correlated dataset
    test_combined, test_fun, test_mar, test_na_fun, test_na_mar = preprocess(fundamental_df, market_df)
    test = reorder(test_combined)
#     test_y = train["return"]
    test = test.drop(columns=["date_time",'stock_id','day'])
    test.to_csv("testing1.csv")
    print(test)
#     test = test.fillna(0)
    # make your prediction Y here and replace the following four rows
#     l = []
#     for idx in range(54):
#         l.append(random.random())
#     predict_ds = pd.Series(l)
    predict_ds = pd.Series(reg.predict(test.iloc[:54]))
    env.input_prediction(predict_ds)  # upload your predicted Y

       turnoverRatio  transactionAmount   pe_ttm       pe      pb      ps  \
0             1.1718             9290.0  63.6900  63.6900  1.6398  1.5477   
1             0.5027             9539.0  33.1536  33.1536  5.2277  3.3677   
2             0.5060             9765.0  36.6197  36.6197  6.0413  4.4686   
...              ...                ...      ...      ...     ...     ...   
37797            NaN                NaN      NaN      NaN     NaN     NaN   
37798            NaN                NaN      NaN      NaN     NaN     NaN   
37799            NaN                NaN      NaN      NaN     NaN     NaN   

            pcf  open_mean  close_mean  high_mean   low_mean  volume_mean  \
0      -40.9539  12.290448   12.296764  12.317646  12.267632    240548.82   
1      124.7176  29.977220   29.987896  30.078692  29.907798     76884.32   
2       33.3814  23.229222   23.240634  23.284810  23.174366    146867.32   
...         ...        ...         ...        ...        ...          ...  

In [ ]:
pd.Series(predict_ds)

In [ ]:
import pandas as pd
print(pd.read_csv('/kaggle/working/submission.csv'))